In [48]:
from pysal.model import spreg
import pandas as pd

data = pd.read_csv("!Arc.csv")
data = data.dropna()



X = data['deprived in three']
print(X)
X1 = data['L1 L2 and L3']
print(X1)

X_train = pd.concat([X,X1], axis=1)
y = data.iloc[:,3]
print(X_train)
#print(y)

0      0.530421
2      2.221004
3      5.777008
4      4.780220
5      1.682692
         ...   
341    3.485086
342    3.698311
343    4.858412
344    2.682990
345    3.105590
Name: deprived in three, Length: 323, dtype: float64
0      27.681307
2      17.326969
3      13.921360
4       9.510619
5      18.564356
         ...    
341    12.708102
342    15.738375
343     9.280742
344    20.750738
345    18.285173
Name: L1 L2 and L3, Length: 323, dtype: float64
     deprived in three  L1 L2 and L3
0             0.530421     27.681307
2             2.221004     17.326969
3             5.777008     13.921360
4             4.780220      9.510619
5             1.682692     18.564356
..                 ...           ...
341           3.485086     12.708102
342           3.698311     15.738375
343           4.858412      9.280742
344           2.682990     20.750738
345           3.105590     18.285173

[323 rows x 2 columns]


In [47]:
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

min = 999
mini = 0
minj = 0
#df = data.iloc[:,8:]
for i in range(4):
    for j in range(8):
        A = data.iloc[:,18+i]
        B = data.iloc[:,8+j]
        df = pd.concat([A, B], axis=1)
        name = df.columns
        QW = np.matrix(df)
        VIF_list = [variance_inflation_factor(QW, i) for i in range(QW.shape[1])]
        max_VIF = max(VIF_list)
        if max_VIF < min:
            mini = i
            minj = j
            min = max_VIF
            print(min)

A = data.iloc[:,18+mini]
B = data.iloc[:,8+minj]
df = pd.concat([A, B], axis=1)
name = df.columns
QW = np.matrix(df)
VIF_list = [variance_inflation_factor(QW, i) for i in range(QW.shape[1])]
VIF = pd.DataFrame({'feature':name,"VIF":VIF_list})
print(VIF)
        

8.425925902308508
6.981638787889896
4.746992436384676
3.4862934559392054
2.3450750800949343
2.067093121453053
1.5274287844290075
             feature       VIF
0  deprived in three  1.527429
1       L1 L2 and L3  1.527429


In [49]:
m1 = spreg.OLS(
    # Dependent variable
    y.values,
    # Independent variables
    X_train.values,
    # Dependent variable name
    # Independent variable name
    name_x=['HDI', 'NS-SeC']
)

print(m1.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :     dep_var                Number of Observations:         323
Mean dependent var  :     10.3221                Number of Variables   :           3
S.D. dependent var  :      2.8577                Degrees of Freedom    :         320
R-squared           :      0.4494
Adjusted R-squared  :      0.4460
Sum squared residual:    1447.810                F-statistic           :    130.5985
Sigma-square        :       4.524                Prob(F-statistic)     :   3.406e-42
S.E. of regression  :       2.127                Log likelihood        :    -700.592
Sigma-square ML     :       4.482                Akaike info criterion :    1407.184
S.E of regression ML:      2.1172                Schwarz criterion     :    1418.517

-----------------------------------------------------------------------------

In [50]:
import pyproj
bng=pyproj.Proj(init='epsg:27700')
wgs84 = pyproj.Proj(init='epsg:4326')
GEO = pd.read_csv("!Dep_fixed1.csv")
GEO = GEO.dropna()

for i in  range(len(GEO)):
    GEO.iloc[i,12] = pyproj.transform(bng, wgs84, GEO.iloc[i,12], GEO.iloc[i,13])[0]
    GEO.iloc[i,13] = pyproj.transform(bng, wgs84, GEO.iloc[i,12], GEO.iloc[i,13])[1]

from libpysal.weights import KNN
import geopandas as gpd
GEO["geometry"] = gpd.points_from_xy(GEO['x'], GEO['y'])

c:\Users\1\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\1\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\1\AppData\Local\Temp\ipykernel_2156\138041753.py:8: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  GEO.iloc[i,12] = pyproj.tra

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:         323
Mean dependent var  :     10.3221                Number of Variables   :           4
S.D. dependent var  :      2.8577                Degrees of Freedom    :         319
Pseudo R-squared    :      0.4582
Spatial Pseudo R-squared:  0.4496

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      11.0946356       2.0041199       5.5359141       0.0000000
                 HDI       0.1862685       0.0731360       2.5468785       0.0108691
              NS-SeC      -0.1651348       0.0297781      -

In [76]:
w = KNN.from_dataframe(GEO, k=3)
m3 = spreg.GM_Lag(y.values, X_train.values, w=w, name_x =['HDI', 'NS-SeC'])
print(m3.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:         323
Mean dependent var  :     10.3221                Number of Variables   :           4
S.D. dependent var  :      2.8577                Degrees of Freedom    :         319
Pseudo R-squared    :      0.4659
Spatial Pseudo R-squared:  0.4518

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       9.8358252       1.8143363       5.4211698       0.0000001
                 HDI       0.1987629       0.0721608       2.7544446       0.0058792
              NS-SeC      -0.1472626       0.0302654      -

In [77]:
from sklearn.metrics import mean_squared_error as mse

mses = pd.Series({'OLS': mse(y, m1.predy.flatten()),
                  'Lag': mse(y, m3.predy_e)
                    })
mses.sort_values()

Lag    4.463962
OLS    4.482383
dtype: float64